## Qdrant
Qdrant is an open-source vector database optimized for storing and querying high-dimensional vector embeddings, commonly used in machine learning and AI applications—especially those involving semantic search, recommendation systems, and retrieval-augmented generation (RAG).

### 🚀 What is Qdrant?
At its core, Qdrant (Quantum Data Rant) is a high-performance vector similarity search engine that:

- Stores data as vectors (like sentence embeddings or image embeddings)

- Enables fast nearest neighbor search based on vector similarity

- Supports hybrid search (vector + filter conditions)

- Provides a RESTful API and gRPC, with official clients for Python, JS, etc.

- Designed to integrate well with AI models, especially LLMs and embeddings

### 💡 When & Why Use Qdrant?
Use Qdrant when you need:

- Semantic search (search by meaning instead of keywords)

- Question answering systems

- RAG pipelines (for LLMs like ChatGPT)

- Clustering and recommendation based on vector proximity

- Handling millions of records efficiently with approximate nearest neighbor (ANN) search

In [30]:
!pip install -q -U langchain langchain-openai langchain-qdrant langchain-community qdrant_client openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [38]:
from langchain.vectorstores import qdrant
from langchain_qdrant import QdrantVectorStore
# from langchain.embeddings.openai import OpenAIEmbeddings  # deprecated
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient, models
import os
from google.colab import userdata

**IMPORTANT NOTE!!!** In order to safely use the api keys and secrets on Google Colab, it is good practice and highly recommended to manually insert them (rather than hardcoding).

**Avoid**...
```
api_key = "sk-abc123..."  # Hardcoded   ❌
```

Below you can how to retrieve the secret you have already inserted into your Google Colab acc.:

In [39]:
# Set qdrant api key via colab secrets
os.environ["QDRANT_API_KEY"] = userdata.get('qdrant-api-key')
# Set qdrant host
os.environ["QDRANT_HOST"] = "https://ff95ca73-34db-4c16-8655-f9cd97f62ec9.europe-west3-0.gcp.cloud.qdrant.io:6333"

In [40]:
# Create a qdrant client
client = QdrantClient(
    url=os.environ["QDRANT_HOST"],
    api_key=os.environ["QDRANT_API_KEY"]
)

In [42]:
# Create collections

os.environ["COLLECTION_NAME"] = "my_collection"

client.create_collection(
    collection_name=os.getenv("COLLECTION_NAME"),
    vectors_config=models.VectorParams(size=1536,   # 768 for instructor-x1, 1536 for OpenAI
                                      distance=models.Distance.COSINE),
)

# to delete the collection, uncomment the line below
# client.delete_collection(collection_name=os.getenv("COLLECTION_NAME"))

True

You can check whether the client has been properly established or not simply through **Thunder Client** for instance. Here’s a very brief step-by-step guide to sending requests using Thunder Client in VS Code:

1. Install Thunder Client
  - Go to the Extensions tab in VS Code
  - Search for Thunder Client and install it

2. Open Thunder Client

  - Click the Thunder Client icon in the sidebar

3. Create a New Request

  - Click “New Request”
  - Choose method: GET, POST, etc.
  - Enter the request URL

4. Set Headers / Body (if needed)

  - Use the Headers tab to add content-type, auth, etc.
  - Use the Body tab for POST, PUT, etc. requests (JSON, form-data, etc.)

5. Send Request

  - Click the “Send” button
  - View response in the bottom panel

Done! 🚀

In [43]:
# Define a function to create chunks
from langchain.text_splitter import CharacterTextSplitter

# Define function
def create_chunks(text):
  # split the text intor chunks
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=500,
      chunk_overlap=50,
      length_function=len
  )
  chunks = text_splitter.split_text(text)

  return chunks

In [44]:
# Read data and create chunks

with open("./of-mice-and-men.txt") as f:
  text = f.read()

texts = create_chunks(text)

In [45]:
print(len(texts))

4


In [46]:
print(texts[0])

In "Of Mice and Men," George tells Lennie about their shared dream of owning a small farm where Lennie can care for rabbits. This dream serves as a source of hope and comfort for Lennie, a mentally disabled man, and also as a shared ambition for George and Lennie. The rabbits specifically represent the peaceful and fulfilling life they hope to create together, a stark contrast to the harsh realities of their current working lives.
Elaboration:
The Dream:


In [47]:
# Count how many points (vectors) exist
count = client.count(collection_name="my_collection", exact=True)
print(f"Total vectors: {count.count}")

Total vectors: 0


In [48]:
# Get info about a specific collection
collection_info = client.get_collection("my_collection")
print(collection_info)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None,

In [49]:
# Create vector store

# Set openai api key via colab secrets
os.environ["OPENAI_API_KEY"] = userdata.get('openai-api-key')

# Initialize the embedding model
embeddings = OpenAIEmbeddings()


# check if the vectore store has already been loaded
# if not load the vector store
if count.count == 0:
  doc_store = QdrantVectorStore.from_texts(
      texts,
      embeddings,
      url=os.environ["QDRANT_HOST"],
      api_key=os.environ["QDRANT_API_KEY"],
      collection_name="my_collection"
  )
  print("Vector store created!")
else:
  doc_store = QdrantVectorStore.from_existing_collection(
      embeddings=embeddings,
      collection_name="my_collection",
      url=os.environ["QDRANT_HOST"],
      api_key=os.environ["QDRANT_API_KEY"],
  )
  print("Vector store loaded!")

Vector store created!


In [50]:
# Count how many points (vectors) exist
count = client.count(collection_name="my_collection", exact=True)
print(f"Total vectors: {count.count}")

Total vectors: 4


In [51]:
# Implement QA chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Initialize the QA chain
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=doc_store.as_retriever()
)

In [52]:
# Make a query

query = "What provides George with a motivation to work hard?"

response = qa.run(query)
print(response)

<ipython-input-52-4d6e60f2492b>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run(query)


 The dream of owning a farm and having rabbits provides George with a motivation to work hard and find a place for themselves.
